# Подключаем диск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Установка библиотек

In [ ]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 26 kB/s 


# Import библиотек

In [ ]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import time
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import lightgbm

from statsmodels.tsa.seasonal import seasonal_decompose
# holt winters 
# single exponential smoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
# double and triple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from scipy.optimize import curve_fit
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Загрузка файлов

In [ ]:
%%time
train = pd.read_csv('/content/drive/MyDrive/all_cups/train.csv', parse_dates=['date'], sep=';', dayfirst=True)
train = train.replace(r'\s+','', regex=True)
train = train.replace(r',','.', regex=True)
train.iloc[:,4:] = train.iloc[:,4:].apply(lambda x: pd.to_numeric(x))
#train = train.sort_values('date')

test = pd.read_csv('/content/drive/MyDrive/all_cups/test.csv', parse_dates=['date'], dayfirst=True)
#test = test.sort_values('date')

sample_submission = pd.read_csv('/content/drive/MyDrive/all_cups/sample_submission.csv', parse_dates=['date'], dayfirst=True)
sample_submission_clear = pd.read_csv('/content/drive/MyDrive/all_cups/sample_submission.csv')

#sample_submission = sample_submission.sort_values('date')

CPU times: user 25.2 s, sys: 738 ms, total: 26 s
Wall time: 28.1 s


# Работа с train. Смотрим тип колонок, количество пропущенных значений по каждой колонке, количество уникальных регионов, октмо, окато, количество дней в датарейме. Лучше работать с октмо ил окато как метками, потому что в пределах одного из регионов (21 регион) есть два разных октмо(окато)

In [ ]:
train.describe()

,region,oktmo,okato,pasta,legumes,bread,flour,rice,groats,potatoes,сucumbers_tomatoes,vegetables,roots,сabbage,fruit,sugar,candy,biscuits,mutton,beef,chicken,pork,fish,herring,curd,sour_creame,butter,milk_kefir,cheese,egg,margarine,oil,salt,spice,tea,cpi_1,cpi_2,cpi_3,pasta_value,legumes_value,bread_value,flour_value,rice_value,groats_value,potatoes_value,сucumbers_tomatoes_value,vegetables_value,roots_value,сabbage_value,fruit_value,sugar_value,candy_value,biscuits_value,mutton_value,beef_value,chicken_value,pork_value,fish_value,herring_value,curd_value,sour_creame_value,butter_value,milk_kefir_value,cheese_value,egg_value,margarine_value,oil_value,salt_value,spice_value,tea_value,ai92,ai95,ai98,dt,ai92_value,ai95_value,ai98_value,dt_value
count,69785.000000,6.978500e+04,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.00000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,6.978500e+04,69785.000000,6.978500e+04
mean,43.376471,5.272353e+10,5.269812e+10,77.904869,92.710147,69.847682,33.101858,73.462862,55.406913,29.008098,133.113410,33.009872,45.031562,27.932438,95.512142,38.343736,294.614938,172.014045,85.979669,325.724765,162.727086,244.751940,331.241146,220.02161,277.303335,202.969405,511.184483,56.946363,491.863814,5.545663,99.021979,86.744528,18.976997,32.942681,76.253347,5404.191011,4606.835339,5444.802231,3.054102e+04,6671.616921,86029.919475,20498.777701,8738.898960,19271.921336,4.146411e+04,30532.491654,19573.245125,16977.963710,15420.424816,7.793083e+04,4.213574e+04,14852.781347,16687.611285,114.152417,2299.863597,57537.825275,13974.507770,1549.703997,2734.145146,10769.456032,11500.723569,5281.055508,1.243611e+05,12372.103317,4.985656e+05,620.133219,20504.957389,7356.908760,35726.172142,32645.479710,42.453241,45.884147,38.784398,47.147889,5.433888e+05,4.090330e+05,4180.121398,1.569221e+05
std,25.211745,2.885514e+10,2.885574e+10,19.458304,29.405926,16.615544,8.871295,19.008159,16.038420,13.864963,75.861292,17.858268,21.570975,16.802193,38.791944,12.371828,81.581619,41.308993,171.785272,123.551169,37.401066,76.329525,166.753402,75.11401,75.357268,78.522210,107.648849,13.365877,86.655124,1.385727,43.200517,21.030084,12.756176,7.987164,14.945813,1238.985598,1048.229812,1345.966941,4.554039e+04,10192.550709,107744.426679,26595.151036,12372.655236,26590.041851,8.267685e+04,54202.605776,33810.828397,34654.221061,24182.106349,1.336904e+05,5.050679e+04,18822.756145,20427.190910,356.535098,4711.957142,81037.374495,19156.329547,3642.047198,3927.216911,20001.273328,17082.562639,7490.922732,1.624299e+05,21260.614669,8.171275e+05,827.936838,25488.297412,9252.952176,46713.017029,46486.810587,4.790249,4.926033,31.764617,6.543259,6.178983e+05,6.624550e+05,10237.652229,2.514019e+05
min,1.000000,1.000000e+09,1.000000e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,

### проверим количество записей в датафрейме, чтобы соответсвовало уникальным значениям * кол-во дней

In [ ]:
print('region unique: ', train['region'].nunique(), 'okato unique: ', train['okato'].nunique(),
      'oktmo unique: ', train['oktmo'].nunique())
print(train['date'].max() - train['date'].min())
# +1 чтобы учитывался и первый, и последний день
print(train['region'].nunique()*(train['date'].max() - train['date'].min()).days,
      train['okato'].nunique()*(train['date'].max() - train['date'].min()).days,
      train['oktmo'].nunique()*(train['date'].max() - train['date'].min()).days,
      len(train))

region unique:  84 okato unique:  85 oktmo unique:  85
820 days 00:00:00
68880 69700 69700 69785


### Пропусков нет

In [ ]:
for col in train.columns:
    pct_missing = np.mean(train[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100,3)))

region - 0.0%
oktmo - 0.0%
okato - 0.0%
date - 0.0%
pasta - 0.0%
legumes - 0.0%
bread - 0.0%
flour - 0.0%
rice - 0.0%
groats - 0.0%
potatoes - 0.0%
сucumbers_tomatoes - 0.0%
vegetables - 0.0%
roots - 0.0%
сabbage - 0.0%
fruit - 0.0%
sugar - 0.0%
candy - 0.0%
biscuits - 0.0%
mutton - 0.0%
beef - 0.0%
chicken - 0.0%
pork - 0.0%
fish - 0.0%
herring - 0.0%
curd - 0.0%
sour_creame - 0.0%
butter - 0.0%
milk_kefir - 0.0%
cheese - 0.0%
egg - 0.0%
margarine - 0.0%
oil - 0.0%
salt - 0.0%
spice - 0.0%
tea - 0.0%
cpi_1 - 0.0%
cpi_2 - 0.0%
cpi_3 - 0.0%
pasta_value - 0.0%
legumes_value - 0.0%
bread_value - 0.0%
flour_value - 0.0%
rice_value - 0.0%
groats_value - 0.0%
potatoes_value - 0.0%
сucumbers_tomatoes_value - 0.0%
vegetables_value - 0.0%
roots_value - 0.0%
сabbage_value - 0.0%
fruit_value - 0.0%
sugar_value - 0.0%
candy_value - 0.0%
biscuits_value - 0.0%
mutton_value - 0.0%
beef_value - 0.0%
chicken_value - 0.0%
pork_value - 0.0%
fish_value - 0.0%
herring_value - 0.0%
curd_value - 0.0%
sour_cr

### Есть отрицательные значения

In [ ]:
for col in train.columns:
  if col == 'date':
    continue
  pct_missing = np.mean(train[col] < 0)
  print('{} - {}%'.format(col, round(pct_missing*100,3)))

In [ ]:
train_abs = train.copy()
train_abs[train_abs.columns[4:]] = train_abs[train_abs.columns[4:]].apply(np.abs)

In [ ]:
train_abs.describe()

,region,oktmo,okato,pasta,legumes,bread,flour,rice,groats,potatoes,сucumbers_tomatoes,vegetables,roots,сabbage,fruit,sugar,candy,biscuits,mutton,beef,chicken,pork,fish,herring,curd,sour_creame,butter,milk_kefir,cheese,egg,margarine,oil,salt,spice,tea,cpi_1,cpi_2,cpi_3,pasta_value,legumes_value,bread_value,flour_value,rice_value,groats_value,potatoes_value,сucumbers_tomatoes_value,vegetables_value,roots_value,сabbage_value,fruit_value,sugar_value,candy_value,biscuits_value,mutton_value,beef_value,chicken_value,pork_value,fish_value,herring_value,curd_value,sour_creame_value,butter_value,milk_kefir_value,cheese_value,egg_value,margarine_value,oil_value,salt_value,spice_value,tea_value,ai92,ai95,ai98,dt,ai92_value,ai95_value,ai98_value,dt_value
count,69785.000000,6.978500e+04,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.00000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,6.978500e+04,69785.000000,6.978500e+04
mean,43.376471,5.272353e+10,5.269812e+10,77.904869,92.710147,69.847682,33.101858,73.462862,55.406913,29.008098,133.113410,33.009872,45.031562,27.932438,95.512142,38.343736,294.614938,172.014045,85.979669,325.724765,162.727086,244.751940,331.241146,220.02161,277.303335,202.969405,511.184483,56.946363,491.863814,5.545663,99.021979,86.744528,18.976997,32.942681,76.253347,5404.191011,4606.835339,5444.802231,3.054102e+04,6671.616921,86029.919475,20498.777701,8738.898960,19271.921336,4.146411e+04,30532.491654,19573.245125,16978.033650,15420.424816,7.793084e+04,4.213574e+04,14852.781347,16687.611285,114.152417,2299.863597,57537.825275,13974.509769,1551.802828,2734.149106,10769.456032,11500.723569,5281.055508,1.243611e+05,12372.103317,4.985656e+05,620.133219,20504.957389,7356.908760,35726.172142,32645.479710,42.453241,45.884147,38.784398,47.147889,5.433888e+05,4.090330e+05,4180.121398,1.569221e+05
std,25.211745,2.885514e+10,2.885574e+10,19.458304,29.405926,16.615544,8.871295,19.008159,16.038420,13.864963,75.861292,17.858268,21.570975,16.802193,38.791944,12.371828,81.581619,41.308993,171.785272,123.551169,37.401066,76.329525,166.753402,75.11401,75.357268,78.522210,107.648849,13.365877,86.655124,1.385727,43.200517,21.030084,12.756176,7.987164,14.945813,1238.985598,1048.229812,1345.966941,4.554039e+04,10192.550709,107744.426679,26595.151036,12372.655236,26590.041851,8.267685e+04,54202.605776,33810.828397,34654.186795,24182.106349,1.336904e+05,5.050679e+04,18822.756145,20427.190910,356.535098,4711.957142,81037.374495,19156.328089,3641.153410,3927.214154,20001.273328,17082.562639,7490.922732,1.624299e+05,21260.614669,8.171275e+05,827.936838,25488.297412,9252.952176,46713.017029,46486.810587,4.790249,4.926033,31.764617,6.543259,6.178983e+05,6.624550e+05,10237.652229,2.514019e+05
min,1.000000,1.000000e+09,1.000000e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,

### Есть значения равные 0. Для 'mutton', 'mutton_value' надо прогнозировать медиану скорее всего.

In [ ]:
for col in train.columns:
  if col == 'date':
    continue
  pct_missing = np.mean(train[col] == 0)
  print('{} - {}%'.format(col, round(pct_missing*100,3)))

### удаление аномалий

In [ ]:
def clear_anomal_IQR(col):
  Q1 = col.quantile(0.07)
  Q3 = col.quantile(0.93)
  IQR = Q3 - Q1
  delete_rows = col[(col < Q1 - 1.5 * IQR) | (col > Q3 + 1.5 * IQR)].index
  clear_col = col.drop(delete_rows, axis=0)

  return clear_col

In [ ]:
train_abs.describe()

,region,oktmo,okato,pasta,legumes,bread,flour,rice,groats,potatoes,сucumbers_tomatoes,vegetables,roots,сabbage,fruit,sugar,candy,biscuits,mutton,beef,chicken,pork,fish,herring,curd,sour_creame,butter,milk_kefir,cheese,egg,margarine,oil,salt,spice,tea,cpi_1,cpi_2,cpi_3,pasta_value,legumes_value,bread_value,flour_value,rice_value,groats_value,potatoes_value,сucumbers_tomatoes_value,vegetables_value,roots_value,сabbage_value,fruit_value,sugar_value,candy_value,biscuits_value,mutton_value,beef_value,chicken_value,pork_value,fish_value,herring_value,curd_value,sour_creame_value,butter_value,milk_kefir_value,cheese_value,egg_value,margarine_value,oil_value,salt_value,spice_value,tea_value,ai92,ai95,ai98,dt,ai92_value,ai95_value,ai98_value,dt_value
count,69785.000000,6.978500e+04,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.00000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,69785.000000,6.978500e+04,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,69785.000000,6.978500e+04,6.978500e+04,69785.000000,6.978500e+04
mean,43.376471,5.272353e+10,5.269812e+10,77.904869,92.710147,69.847682,33.101858,73.462862,55.406913,29.008098,133.113410,33.009872,45.031562,27.932438,95.512142,38.343736,294.614938,172.014045,85.979669,325.724765,162.727086,244.751940,331.241146,220.02161,277.303335,202.969405,511.184483,56.946363,491.863814,5.545663,99.021979,86.744528,18.976997,32.942681,76.253347,5404.191011,4606.835339,5444.802231,3.054102e+04,6671.616921,86029.919475,20498.777701,8738.898960,19271.921336,4.146411e+04,30532.491654,19573.245125,16978.033650,15420.424816,7.793084e+04,4.213574e+04,14852.781347,16687.611285,114.152417,2299.863597,57537.825275,13974.509769,1551.802828,2734.149106,10769.456032,11500.723569,5281.055508,1.243611e+05,12372.103317,4.985656e+05,620.133219,20504.957389,7356.908760,35726.172142,32645.479710,42.453241,45.884147,38.784398,47.147889,5.433888e+05,4.090330e+05,4180.121398,1.569221e+05
std,25.211745,2.885514e+10,2.885574e+10,19.458304,29.405926,16.615544,8.871295,19.008159,16.038420,13.864963,75.861292,17.858268,21.570975,16.802193,38.791944,12.371828,81.581619,41.308993,171.785272,123.551169,37.401066,76.329525,166.753402,75.11401,75.357268,78.522210,107.648849,13.365877,86.655124,1.385727,43.200517,21.030084,12.756176,7.987164,14.945813,1238.985598,1048.229812,1345.966941,4.554039e+04,10192.550709,107744.426679,26595.151036,12372.655236,26590.041851,8.267685e+04,54202.605776,33810.828397,34654.186795,24182.106349,1.336904e+05,5.050679e+04,18822.756145,20427.190910,356.535098,4711.957142,81037.374495,19156.328089,3641.153410,3927.214154,20001.273328,17082.562639,7490.922732,1.624299e+05,21260.614669,8.171275e+05,827.936838,25488.297412,9252.952176,46713.017029,46486.810587,4.790249,4.926033,31.764617,6.543259,6.178983e+05,6.624550e+05,10237.652229,2.514019e+05
min,1.000000,1.000000e+09,1.000000e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,

In [ ]:
%%time
train_abs_clear = train_abs.copy()

for oktmo in tqdm(train_abs_clear['oktmo'].unique()):
  train_abs_clear.loc[train_abs_clear['oktmo'] == oktmo, train_abs_clear.columns[train_abs_clear.columns.str.contains(r"_value")]] =\
   train_abs_clear[train_abs_clear['oktmo'] == oktmo][train_abs_clear.columns[train_abs_clear.columns.str.contains(r"_value")]].apply(clear_anomal_IQR)

  0%|          | 0/85 [00:00<?, ?it/s]

CPU times: user 12.3 s, sys: 174 ms, total: 12.4 s
Wall time: 12.5 s


# Проверим test

In [ ]:
print('region unique: ', test['region'].nunique(), 'okato unique: ', test['okato'].nunique(),
      'oktmo unique: ', test['oktmo'].nunique())
print(test['date'].max() - test['date'].min())
print(test['region'].nunique()*(test['date'].max() - test['date'].min()).days,
      test['okato'].nunique()*(test['date'].max() - test['date'].min()).days,
      test['oktmo'].nunique()*(test['date'].max() - test['date'].min()).days,
      len(test))

region unique:  84 okato unique:  85 oktmo unique:  85
90 days 00:00:00
7560 7650 7650 7735


## Для test аналогичная ситуация и опять для 21 региона.

# Добавим данные о праздниках из https://github.com/d10xa/holidays-calendar

In [ ]:
year_of_holidays = ['2021', '2020', '2019']
for year in year_of_holidays:
  with open ('/content/drive/MyDrive/all_cups/consultant{}.json'.format(year)) as file:
    data = json.load(file)
    df = pd.json_normalize(data)
    train_abs_clear.loc[train_abs_clear['date'].isin(df['holidays'][0]), 'is_holidays'] = 1
    train_abs_clear['is_holidays'].fillna(0, inplace=True)
    train_abs_clear.loc[train_abs_clear['date'].isin(df['preholidays'][0]), 'is_preholidays'] = 1
    train_abs_clear['is_preholidays'].fillna(0, inplace=True)
    try:
      train_abs_clear.loc[train_abs_clear['date'].isin(df['nowork'][0]), 'is_nowork'] = 1
      train_abs_clear['is_nowork'].fillna(0, inplace=True)
    except:
      pass

    test.loc[test['date'].isin(df['holidays'][0]), 'is_holidays'] = 1
    test['is_holidays'].fillna(0, inplace=True)
    test.loc[test['date'].isin(df['preholidays'][0]), 'is_preholidays'] = 1
    test['is_preholidays'].fillna(0, inplace=True)
    try:
      test.loc[test['date'].isin(df['nowork'][0]), 'is_nowork'] = 1
      test['is_nowork'].fillna(0, inplace=True)
    except:
      pass

# Коронавирус https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_COVID-19_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8

In [ ]:
data = pd.date_range('03-30-2020 00:00:00', '07-01-2020 00:00:00', freq='1d')
train_abs_clear['COVID'] = train_abs_clear['date'].isin(data)*1
test['COVID'] = test['date'].isin(data)*1

In [ ]:
train_abs_clear_COVID = train_abs_clear.copy()
train_abs_clear_COVID = train_abs_clear_COVID.drop(train_abs_clear[train_abs_clear['COVID'] == 1].index, axis=0)

# Функция для генерации фичей и разбиения на трейн и тест

In [ ]:
def prepareData(DataFrame, col, lag_start=-1, lag_end=-1, test_size=-1):
      data = DataFrame.copy()

      # Добавляем лаги
      for i in range(lag_start, lag_end):
        for oktmo in data['oktmo'].unique():
           data.loc[data['oktmo'] == oktmo, "lag_{}".format(i)] =  data[data['oktmo'] == oktmo][col].shift(i)

      data['day'] = data.date.dt.day
      data['week'] = data.date.dt.week
      data['month'] = data.date.dt.month
      data['year'] = data.date.dt.year
      #data['quarter'] = data.date.dt.quarter

      data['is_summer'] = data.month.isin([6,7,8])*1
      data['is_winter'] = data.month.isin([12,1,2])*1
      data['is_autumn'] = data.month.isin([9,10,11])*1
      data['is_spring'] = data.month.isin([3,4,5])*1


      data['weekday'] = data.date.dt.weekday
      data['is_weekend'] = data.weekday.isin([5,6])*1

      data['weekofyear'] = data.date.dt.weekofyear
      data['dayofyear'] = data.date.dt.dayofyear

      data = data.reset_index()

    # разбиваем весь датасет на тренировочную и тестовую выборку
      if test_size != -1:

        X_train = data.loc[:(len(data)-test_size-1)]

        y_train = data.loc[:(len(data)-test_size-1)][col]

        X_test = data.loc[len(data)-test_size:]

        y_test = data.loc[len(data)-test_size:][col]
        return X_train, X_test, y_train, y_test
    
      else:
        return data_reset_index


# лучшая попытка 14.08.2021

In [ ]:
train_final = train_abs_clear_COVID.dropna().sort_values('date')
train_test = pd.concat([train_final, test])
print('train: ', train_final.shape, 'test: ', test.shape,
      'train_test: ', train_test.shape)
#train_test_without_mutton = train_test.drop(['mutton', 'mutton_value'], axis=1)

train:  (59182, 83) test:  (7735, 83) train_test:  (66917, 83)


In [ ]:
%%time
for col in tqdm(train_test.columns[4:79]):
  X_train_data, X_test_data, y_train, y_test = prepareData(
      train_test[['oktmo', 'region', col, 'is_holidays', 'is_preholidays', 'is_nowork', 'date']], col,
       test_size=len(test), lag_start=-1, lag_end=-1)
  X_train = X_train_data.drop(['date', 'index', col] , axis=1)
  X_test = X_test_data.drop(['date', 'index', col], axis=1)
  tic = time.time()
  CBR = CatBoostRegressor(verbose=False, random_state=42)
  CBR.fit(X_train, y_train)
  y_pred_CBR = CBR.predict(X_test)
  y_pred_CBR[y_pred_CBR > X_train_data[col].max()] = X_train_data[col].max()
  y_pred_CBR[y_pred_CBR < X_train_data[col].min()] = X_train_data[col].min()

  toc = time.time()
  print('time models {}: {} sec'.format(col, (toc - tic)))
  print('mae for {}: '.format(col), np.sum(abs(y_pred_CBR - train_test[col].mean()))/len(y_pred_CBR))

  sample_submission[col] = y_pred_CBR


  0%|          | 0/75 [00:00<?, ?it/s]

time models pasta: 10.134302854537964 sec
mae for pasta:  14.301776718183845
time models legumes: 9.918015956878662 sec
mae for legumes:  20.371089464126097
time models bread: 10.097876071929932 sec
mae for bread:  11.717911707569728
time models flour: 10.043300151824951 sec
mae for flour:  5.9037991148981455
time models rice: 9.965951919555664 sec
mae for rice:  13.357442417924915
time models groats: 10.003041982650757 sec
mae for groats:  16.00661037473398
time models potatoes: 10.052978754043579 sec
mae for potatoes:  13.74351316978271
time models сucumbers_tomatoes: 9.888831377029419 sec
mae for сucumbers_tomatoes:  39.272924081397704
time models vegetables: 10.060259580612183 sec
mae for vegetables:  9.216673205886618
time models roots: 9.99098253250122 sec
mae for roots:  21.683064441086195
time models сabbage: 10.023863077163696 sec
mae for сabbage:  8.510270656474026
time models fruit: 9.941944122314453 sec
mae for fruit:  17.949107543172868
time models sugar: 10.01971411705017

In [ ]:
df = sample_submission.copy()

In [ ]:
df.describe()

,region,oktmo,okato,pasta,legumes,bread,flour,rice,groats,potatoes,сucumbers_tomatoes,vegetables,roots,сabbage,fruit,sugar,candy,biscuits,mutton,beef,chicken,pork,fish,herring,curd,sour_creame,butter,milk_kefir,cheese,egg,margarine,oil,salt,spice,tea,cpi_1,cpi_2,cpi_3,pasta_value,legumes_value,bread_value,flour_value,rice_value,groats_value,potatoes_value,сucumbers_tomatoes_value,vegetables_value,roots_value,сabbage_value,fruit_value,sugar_value,candy_value,biscuits_value,mutton_value,beef_value,chicken_value,pork_value,fish_value,herring_value,curd_value,sour_creame_value,butter_value,milk_kefir_value,cheese_value,egg_value,margarine_value,oil_value,salt_value,spice_value,tea_value,ai92,ai95,ai98,dt,ai92_value,ai95_value,ai98_value,dt_value
count,7735.000000,7.735000e+03,7.735000e+03,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7.735000e+03,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7.735000e+03,7735.000000,7.735000e+03,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7735.000000,7.735000e+03,7.735000e+03,7735.000000,7.735000e+03
mean,43.376471,5.272353e+10,5.269812e+10,88.791868,107.710064,76.201373,36.775231,80.141313,67.983968,41.144635,139.144131,39.338756,64.101766,33.606095,108.145659,46.234997,308.301433,186.373481,106.841815,343.973119,182.998458,259.296818,364.145620,236.912034,288.146343,208.571236,521.365990,60.087649,519.262413,7.182408,119.552316,115.094229,20.568359,38.200205,84.504798,6012.197644,5144.828736,6102.631361,36580.471667,6385.230142,99139.947228,19883.738829,8875.264424,19793.458279,51639.130653,45396.220789,23781.465560,18765.136297,17007.221992,9.826766e+04,48942.521972,16482.249442,20768.052466,163.514160,3320.023416,68446.357325,17526.481918,2190.541962,3443.311092,13209.570606,14260.203065,6627.810823,1.481713e+05,15887.587159,5.634678e+05,608.812413,22087.506656,8563.270715,41198.726287,37715.587061,44.552754,47.772473,36.827270,48.573219,5.782696e+05,4.602250e+05,5197.782042,1.629118e+05
std,25.213194,2.885680e+10,2.885739e+10,19.871389,22.243671,15.860911,7.664734,17.007060,14.225135,10.626033,83.126082,11.208024,17.253847,12.911475,40.177552,10.891594,62.279182,37.962454,166.091015,113.690799,35.807954,67.481266,140.617575,63.594477,71.669872,72.952247,99.038960,12.439158,82.720816,1.479509,41.884440,18.952920,7.586399,8.153715,10.694044,1300.698749,1100.892096,1435.478905,50883.822640,9460.423217,111537.735706,22260.557078,11506.690894,25513.970637,85926.039692,66060.093587,38627.981087,35880.319778,24342.322804,1.541365e+05,48497.883971,18374.658060,22314.636630,441.797751,5968.219465,94284.548952,21676.777081,4507.657427,4047.699580,23153.431378,18885.975652,8925.782274,1.754252e+05,25609.944685,8.652932e+05,869.100742,26026.522201,10030.714761,52083.339121,51764.786147,2.724490,2.388483,21.503770,5.874829,6.598050e+05,7.292998e+05,11258.882084,2.506926e+05
min,1.000000,1.000000e+09,1.000000e+09,14.322644,2.225083,32.149912,0.000000,0.000000,4.775613,0.000000,0.000000,0.000000,2.499199,0.000000,62.636852,0.000000,118.380935,107.403855,0.000000,0.000000,6.724378,0.000000,0.000000,0.000000,0.000000,15.554155,0.000000,30.714349,0.737675,0.882404,0.000000,4.544884,0.000000,20.822570,43.498588,229.261563,224.330143,228.033939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000

In [ ]:
# Заполним нулями те колонки для 6, 20 и 87 региона, которые были нулевые в train
df.loc[df['region'] == 6, df.columns[4:][pd.Series(train[train['region'] == 6][4:].std() == 0).values[3:]]] = 0
df.loc[df['region'] == 20, df.columns[4:][pd.Series(train[train['region'] == 20][4:].std() == 0).values[3:]]] = 0
df.loc[df['region'] == 87, df.columns[4:][pd.Series(train[train['region'] == 87][4:].std() == 0).values[3:]]] = 0

In [ ]:
# Лучшее решение
df.to_csv('df.csv', index=False)

In [ ]:
# Предыдущее лучшее решение без исправления регионов
sample_submission.to_csv('sample_submission14.08_CBR_0.07_0.93__with_drop_COVID_03_30_2020_07.01.2020.csv', index=False)